<a href="https://colab.research.google.com/github/srivatsan88/YouTubeLI/blob/master/Data_Analysis_and_Data_Validation_using_TFX.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from __future__ import print_function
import sys,tempfile, urllib, os

In [32]:
sys.version_info.major

3

In [33]:
!pip install tfx==0.21.0 tensorflow==2.1 tensorboard==2.1


In [0]:
BASE_DIR = tempfile.mkdtemp(prefix='tfx-data')
OUTPUT_FILE = os.path.join(BASE_DIR, 'house_price.csv')

In [0]:
bank_data=urllib.request.urlretrieve('https://raw.githubusercontent.com/srivatsan88/YouTubeLI/master/dataset/train_hprice.csv', OUTPUT_FILE)


In [36]:
import tensorflow_data_validation as tfdv

train_stats = tfdv.generate_statistics_from_csv(data_location=OUTPUT_FILE)

/usr/local/lib/python3.6/dist-packages/tensorflow_data_validation/arrow/arrow_util.py:236: FutureWarning: Calling .data on ChunkedArray is provided for compatibility after Column was removed, simply drop this attribute
  types.FeaturePath([column_name]), column.data.chunk(0), weights):


In [37]:
tfdv.visualize_statistics(train_stats)

In [9]:
schema = tfdv.infer_schema(statistics=train_stats)
tfdv.display_schema(schema=schema)

,Type,Presence,Valency,Domain
Feature name,,,,
'Id',INT,required,,-
'MSSubClass',INT,required,,-
'MSZoning',STRING,required,,'MSZoning'
'LotFrontage',BYTES,required,,-
'LotArea',INT,required,,-
...,...,...,...,...
'MoSold',INT,required,,-
'YrSold',INT,required,,-
'SaleType',STRING,required,,'SaleType'


,Values
Domain,
'MSZoning',"'C (all)', 'FV', 'RH', 'RL', 'RM'"
'Street',"'Grvl', 'Pave'"
'Alley',"'Grvl', 'NA', 'Pave'"
'LotShape',"'IR1', 'IR2', 'IR3', 'Reg'"
'LandContour',"'Bnk', 'HLS', 'Low', 'Lvl'"
'Utilities',"'AllPub', 'NoSeWa'"
'LotConfig',"'Corner', 'CulDSac', 'FR2', 'FR3', 'Inside'"
'LandSlope',"'Gtl', 'Mod', 'Sev'"
'Neighborhood',"'Blmngtn', 'Blueste', 'BrDale', 'BrkSide', 'ClearCr', 'CollgCr', 'Crawfor', 'Edwards', 'Gilbert', 'IDOTRR', 'MeadowV', 'Mitchel', 'NAmes', 'NPkVill', 'NWAmes', 'NoRidge', 'NridgHt', 'OldTown', 'SWISU', 'Sawyer', 'SawyerW', 'Somerst', 'StoneBr', 'Timber', 'Veenker'"


In [0]:
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow_data_validation.statistics import stats_options as options

In [0]:
house_df = pd.read_csv(OUTPUT_FILE)

In [12]:
house_df.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,...,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2003,2003,Gable,CompShg,VinylSd,VinylSd,BrkFace,196.0,Gd,TA,PConc,Gd,TA,No,GLQ,706,Unf,0,150,856,GasA,...,Y,SBrkr,856,854,0,1710,1,0,2,1,3,1,Gd,8,Typ,0,NaN,Attchd,2003.0,RFn,2,548,TA,TA,Y,0,61,0,0,0,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,Gtl,Veenker,Feedr,Norm,1Fam,1Story,6,8,1976,1976,Gable,CompShg,MetalSd,MetalSd,None,0.0,TA,TA,CBlock,Gd,TA,Gd,ALQ,978,Unf,0,284,1262,GasA,...,Y,SBrkr,1262,0,0,1262,0,1,2,0,3,1,TA,6,Typ,1,TA,Attchd,1976.0,RFn,2,460,TA,TA,Y,298,0,0,0,0,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2001,2002,Gable,CompShg,VinylSd,VinylSd,BrkFace,162.0,Gd,TA,PConc,Gd,TA,Mn,GLQ,486,Unf,0,434,920,GasA,...,Y,SBrkr,920,866,0,1786,1,0,2,1,3,1,Gd,6,Typ,1,TA,Attchd,2001.0,RFn,2,608,TA,TA,Y,0,42,0,0,0,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,Corner,Gtl,Crawfor,Norm,Norm,1Fam,2Story,7,5,1915,1970,Gable,CompShg,Wd Sdng,Wd Shng,None,0.0,TA,TA,BrkTil,TA,Gd,No,ALQ,216,Unf,0,540,756,GasA,...,Y,SBrkr,961,756,0,1717,1,0,1,0,3,1,Gd,7,Typ,1,Gd,Detchd,1998.0,Unf,3,642,TA,TA,Y,0,35,272,0,0,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,FR2,Gtl,NoRidge,Norm,Norm,1Fam,2Story,8,5,2000,2000,Gable,CompShg,VinylSd,VinylSd,BrkFace,350.0,Gd,TA,PConc,Gd,TA,Av,GLQ,655,Unf,0,490,1145,GasA,...,Y,SBrkr,1145,1053,0,2198,1,0,2,1,4,1,Gd,9,Typ,1,TA,Attchd,2000.0,RFn,3,836,TA,TA,Y,192,84,0,0,0,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [0]:
house_train, house_eval = train_test_split(house_df, test_size=0.2)

In [23]:
house_eval.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,...,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
839,840,50,RL,70.0,11767,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,Edwards,Norm,Norm,1Fam,1.5Fin,5,6,1946,1995,Gable,CompShg,MetalSd,MetalSd,None,0.0,TA,TA,CBlock,TA,TA,No,BLQ,352,Unf,0,416,768,GasA,...,Y,SBrkr,768,432,0,1200,0,0,1,0,3,1,TA,6,Typ,0,NaN,Detchd,1946.0,Unf,1,240,TA,TA,Y,0,0,0,0,0,0,NaN,NaN,NaN,0,5,2008,WD,Normal,130500
1188,1189,60,RL,68.0,8935,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2002,2002,Gable,CompShg,VinylSd,VinylSd,BrkFace,95.0,Gd,TA,PConc,Gd,TA,No,Unf,0,Unf,0,831,831,GasA,...,Y,SBrkr,831,829,0,1660,0,0,2,1,3,1,Gd,7,Typ,0,NaN,Attchd,2002.0,RFn,2,493,TA,TA,Y,144,68,0,0,0,0,NaN,NaN,NaN,0,7,2009,WD,Normal,195000
1012,1013,70,RL,55.0,10592,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,Crawfor,Norm,Norm,1Fam,2Story,6,7,1923,1996,Hip,CompShg,Wd Sdng,Wd Sdng,None,0.0,TA,Gd,PConc,TA,Fa,No,Unf,0,Unf,0,602,602,GasA,...,Y,SBrkr,900,602,0,1502,0,0,1,1,3,1,Gd,7,Typ,2,TA,Detchd,1923.0,Unf,1,180,TA,TA,Y,96,0,112,0,53,0,NaN,NaN,NaN,0,8,2007,WD,Normal,165000
1384,1385,50,RL,60.0,9060,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,Edwards,Norm,Norm,1Fam,1.5Fin,6,5,1939,1950,Gable,CompShg,WdShing,Wd Shng,None,0.0,TA,TA,BrkTil,TA,TA,Mn,Rec,204,Unf,0,356,560,GasA,...,Y,SBrkr,698,560,0,1258,0,0,1,0,2,1,TA,6,Typ,0,NaN,Detchd,1939.0,Unf,1,280,TA,TA,P,0,0,0,0,0,0,NaN,MnPrv,NaN,0,10,2009,WD,Normal,105000
592,593,20,RL,60.0,6600,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,Mitchel,Norm,Norm,1Fam,1Story,5,8,1982,2003,Gable,CompShg,HdBoard,HdBoard,None,0.0,TA,Gd,PConc,TA,Gd,No,GLQ,816,Unf,0,0,816,GasA,...,Y,SBrkr,816,0,0,816,1,0,1,0,2,1,TA,4,Typ,0,NaN,Detchd,1985.0,Fin,2,816,TA,TA,Y,0,0,0,0,0,0,NaN,NaN,NaN,0,6,2008,WD,Normal,138000


In [25]:
train_df_stats=tfdv.generate_statistics_from_dataframe(house_train,stats_options=options.StatsOptions())

/usr/local/lib/python3.6/dist-packages/tensorflow_data_validation/arrow/arrow_util.py:236: FutureWarning: Calling .data on ChunkedArray is provided for compatibility after Column was removed, simply drop this attribute
  types.FeaturePath([column_name]), column.data.chunk(0), weights):


In [26]:
tfdv.visualize_statistics(train_df_stats)

In [27]:
eval_df_stats=tfdv.generate_statistics_from_dataframe(house_eval,stats_options=options.StatsOptions())

/usr/local/lib/python3.6/dist-packages/tensorflow_data_validation/arrow/arrow_util.py:236: FutureWarning: Calling .data on ChunkedArray is provided for compatibility after Column was removed, simply drop this attribute
  types.FeaturePath([column_name]), column.data.chunk(0), weights):


So far we've visualized dataset individually. It's important that our evaluation data is consistent with our training data, including that it uses the same schema. It's also important that the evaluation data includes examples of roughly the same ranges of values for our numerical features as our training data, so that our coverage of the loss surface during evaluation is roughly the same as during training. The same is true for categorical features. Otherwise, we may have training issues that are not identified during evaluation, because we didn't evaluate part of our loss surface.

Notice in below graph each feature now includes statistics for both the training and evaluation datasets.
Notice that the charts now have both the training and evaluation datasets overlaid, making it easy to compare them.
Notice that the charts now include a percentages view, which can be combined with log or the default linear scales.


In [28]:
tfdv.visualize_statistics(lhs_statistics=eval_df_stats, rhs_statistics=train_df_stats,lhs_name='EVAL_DATASET', rhs_name='TRAIN_DATASET')

In [38]:
eval_anomalies = tfdv.validate_statistics(statistics=eval_df_stats, schema=schema)
tfdv.display_anomalies(eval_anomalies)

,Anomaly short description,Anomaly long description
Feature name,,
'GarageFinish',Column dropped,The feature was present in fewer examples than expected.
'MasVnrType',Column dropped,The feature was present in fewer examples than expected.
'BsmtCond',Column dropped,The feature was present in fewer examples than expected.
'BsmtExposure',Column dropped,The feature was present in fewer examples than expected.
'PoolQC',Column dropped,The feature was not present in any examples.
'Fence',Column dropped,The feature was present in fewer examples than expected.
'GarageType',Column dropped,The feature was present in fewer examples than expected.
'LotFrontage',Column dropped,The feature was present in fewer examples than expected.
'GarageQual',Column dropped,The feature was present in fewer examples than expected.


We can update schema manually in case schema evolves over time or additional metadata needs to be fed in to schema

Validate_statistics in addition to comparing schema can also be used to setup skew compartor and drift compator. Display_anomolies function will indicator features that have skewed or have drifted above the set threshold

In [45]:
sf_skew = tfdv.get_feature(schema, '2ndFlrSF')
sf_skew.skew_comparator.infinity_norm.threshold = 0.001

f_comp =tfdv.get_feature(schema, 'Fence')
f_comp.drift_comparator.infinity_norm.threshold = 0.001

gc_comp =tfdv.get_feature(schema, 'GarageCond')
gc_comp.drift_comparator.infinity_norm.threshold = 0.001

skew_anomalies = tfdv.validate_statistics(eval_df_stats, schema,
                                          previous_statistics=train_df_stats)
tfdv.display_anomalies(skew_anomalies)

,Anomaly short description,Anomaly long description
Feature name,,
'GarageYrBlt',Column dropped,The feature was present in fewer examples than expected.
'MasVnrArea',Column dropped,The feature was present in fewer examples than expected.
'FireplaceQu',Column dropped,The feature was present in fewer examples than expected.
'BsmtFinType1',Column dropped,The feature was present in fewer examples than expected.
'BsmtFinType2',Column dropped,The feature was present in fewer examples than expected.
'BsmtQual',Column dropped,The feature was present in fewer examples than expected.
'GarageCond',Multiple errors,"The feature was present in fewer examples than expected. The Linfty distance between current and previous is 0.0113831 (up to six significant digits), above the threshold 0.001. The feature value with maximum difference is: TA"
'GarageFinish',Column dropped,The feature was present in fewer examples than expected.
'MasVnrType',Column dropped,The feature was present in fewer examples than expected.
